**get model coordinates**

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

coor = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/distan_lat42.58lon-8.8046p3R4Km.csv")
px.set_mapbox_access_token("pk.eyJ1IjoiZ3JhbmFudHVpbiIsImEiOiJja3B4dGU4OTkwMTFmMm9ycnNhMjJvaGJqIn0.VWzx_PkD9A5cSUVsn_ijCA")
px.scatter_mapbox(coor, hover_data=['distance'],lat='lat', lon='lon',color='distance', title="nearest points",
                           color_continuous_scale=px.colors.cyclical.IceFire,)

**get station and sample sd speed mean hour before**

In [2]:
station = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/station_data/coron.csv", usecols = ["time","sddir_o"],parse_dates=["time"]).set_index("time")
print("raw data\n",station.head(24))

#show duplicated
df_duplicate = station[station.index.duplicated()]
print("duplicated")
print(df_duplicate)

#delete duplicated and resample
station = station[~station.index.duplicated(keep='first')]
station = station[station>0]
station = station.resample("H",label="right",closed="right").mean()

print("dir hour before")
print(station.head(4))
print("Variable statistics")
print(station.describe())

raw data
                      sddir_o
time                        
2018-01-01 00:10:00     10.0
2018-01-01 00:20:00      9.0
2018-01-01 00:30:00     11.0
2018-01-01 00:40:00     11.0
2018-01-01 00:50:00     13.0
2018-01-01 01:00:00     12.0
2018-01-01 01:10:00     11.0
2018-01-01 01:20:00     10.0
2018-01-01 01:30:00     11.0
2018-01-01 01:40:00     12.0
2018-01-01 01:50:00     11.0
2018-01-01 02:00:00     13.0
2018-01-01 02:10:00     11.0
2018-01-01 02:20:00     10.0
2018-01-01 02:30:00     11.0
2018-01-01 02:40:00     12.0
2018-01-01 02:50:00     14.0
2018-01-01 03:00:00     15.0
2018-01-01 03:10:00     12.0
2018-01-01 03:20:00     11.0
2018-01-01 03:30:00     11.0
2018-01-01 03:40:00      9.0
2018-01-01 03:50:00      9.0
2018-01-01 04:00:00     11.0
duplicated
                     sddir_o
time                        
2018-03-25 03:00:00     12.0
2018-03-25 03:10:00      5.0
2018-03-25 03:20:00      3.0
2018-03-25 03:30:00      5.0
2018-03-25 03:40:00      6.0
2018-03-25 03:50:00   

**get meteorologic model d0**

In [3]:
#ria forecast d0
met_model = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Coron/met_ model/lat42.58lon-8.8046p3R4KmD0.csv",parse_dates=["time"]).drop(columns=["Unnamed: 0"]).set_index("time")
met_model.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42021 entries, 2018-01-01 01:00:00 to 2023-01-01 23:00:00
Data columns (total 75 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dir0         42021 non-null  float64
 1   snow_prec0   42021 non-null  float64
 2   snowlevel0   42021 non-null  float64
 3   mod0         42021 non-null  float64
 4   wind_gust0   42021 non-null  float64
 5   mslp0        42021 non-null  float64
 6   temp0        42021 non-null  float64
 7   rh0          42021 non-null  float64
 8   visibility0  42021 non-null  float64
 9   lhflx0       42021 non-null  float64
 10  lwflx0       42021 non-null  float64
 11  conv_prec0   42021 non-null  float64
 12  prec0        42021 non-null  float64
 13  swflx0       42021 non-null  float64
 14  shflx0       42021 non-null  float64
 15  cape0        42021 non-null  float64
 16  cin0         42021 non-null  float64
 17  cfh0         42021 non-null  float64
 18  cfl0       

**Concatenate and dropnan**

In [4]:
df_all = pd.concat([station,met_model],axis=1).dropna()
df_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41369 entries, 2018-01-01 01:00:00 to 2022-12-30 23:00:00
Data columns (total 76 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sddir_o      41369 non-null  float64
 1   dir0         41369 non-null  float64
 2   snow_prec0   41369 non-null  float64
 3   snowlevel0   41369 non-null  float64
 4   mod0         41369 non-null  float64
 5   wind_gust0   41369 non-null  float64
 6   mslp0        41369 non-null  float64
 7   temp0        41369 non-null  float64
 8   rh0          41369 non-null  float64
 9   visibility0  41369 non-null  float64
 10  lhflx0       41369 non-null  float64
 11  lwflx0       41369 non-null  float64
 12  conv_prec0   41369 non-null  float64
 13  prec0        41369 non-null  float64
 14  swflx0       41369 non-null  float64
 15  shflx0       41369 non-null  float64
 16  cape0        41369 non-null  float64
 17  cin0         41369 non-null  float64
 18  cfh0       

**LGBMRegressor**

In [5]:
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.sddir_o
X = df_all[df_all.columns[1:]]

MAE = []
for t in range (0,10):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE":MAE,});
df_r.describe()


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.076720 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18216
[LightGBM] [Info] Number of data points in the train set: 37232, number of used features: 72
[LightGBM] [Info] Start training from score 10.570052
machine learning number: 0
            y_test       y_pred         diff
count  4137.000000  4137.000000  4137.000000
mean     10.632225    10.589968    -0.042258
std       5.996484     3.613110     4.747583
min       1.500000     3.132334   -32.159165
25%       5.833333     7.503934    -1.460930
50%       9.500000    10.732348     0.618555
75%      14.000000    13.147468     2.429861
max      50.333333    25.163005    16.970248
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023230 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18216
[LightGBM] [Info] Number of

Mean absolute value all machine learning models


,MAE
count,10.000000
mean,3.111000
std,0.047947
min,3.040000
25%,3.072500
50%,3.120000
75%,3.150000
max,3.170000


**Shapiro test normality MAE samples tested**

In [6]:
from scipy.stats import shapiro

sta, p = shapiro(MAE)
print("\np value:",p)
print("significance level alpha=0.01")

alpha = 0.01
if p>alpha:
  print("Accept null hypothesis: normal distribution variable MAE in the samples")
else:
  print("Reject null Hypothesis. Alternative hypothesis variable MAE no normal distribution in samples")


p value: 0.29021865129470825
significance level alpha=0.01
Accept null hypothesis: normal distribution variable MAE in the samples


**Add time variables**

In [7]:

df_all["hour"] = df_all.index.hour
df_all["month"] = df_all.index.month
df_all["dayofyear"] = df_all.index.dayofyear
df_all["weekofyear"] = df_all.index.isocalendar().week.astype(int)

#numpy randomseed
np.random.seed(1)

# target variable
Y = df_all.sddir_o
X = df_all[df_all.columns[1:]]


MAE_t = []
for t in range (0,10):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, )
  ml_model = LGBMRegressor(n_estimators=200).fit(X_train,y_train)
  y_pred = ml_model.predict(X_test)
  MAE_t.append(round(mean_absolute_error(y_test,y_pred),2))
  print("machine learning number:",t)
  df = pd.DataFrame({"y_test":y_test},index=y_test.index)
  df["y_pred"] = y_pred
  df["diff"] = df["y_pred"]-df["y_test"]
  print(df.describe())
print("\nDifferences last machine learning model")
fig = px.line(df["diff"].sort_index())
fig.show()
print("Mean absolute value all machine learning models")
df_r=pd.DataFrame({"MAE_t":MAE_t,});
df_r.describe()



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025798 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18555
[LightGBM] [Info] Number of data points in the train set: 37232, number of used features: 76
[LightGBM] [Info] Start training from score 10.570052
machine learning number: 0
            y_test       y_pred         diff
count  4137.000000  4137.000000  4137.000000
mean     10.632225    10.578588    -0.053637
std       5.996484     3.642935     4.675255
min       1.500000     3.068861   -35.270903
25%       5.833333     7.477547    -1.492539
50%       9.500000    10.633091     0.622342
75%      14.000000    13.160715     2.437452
max      50.333333    27.885041    16.385041
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025938 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18557
[LightGBM] [Info] Number of

Mean absolute value all machine learning models


,MAE_t
count,10.000000
mean,3.078000
std,0.043665
min,3.000000
25%,3.042500
50%,3.085000
75%,3.117500
max,3.130000


**T test two related samples. This is a test for the null hypothesis that two related samples have identical mean absolute error. Alternative hypothesis the  mean absolute error from the sample with time variables is less than the mean absolute error of from the sample with time variables. Significance level (alpha) = 0.01**

In [8]:
from scipy import stats

alpha = 0.01
sta, p_value = stats.ttest_rel(MAE, MAE_t, alternative="greater")

print("\nMAE without time variables:", np.round(MAE,3))
print("\nMAE with time variables:", np.round(MAE_t,3))
print("\nalpha: ",alpha)
print("\np_value point: ",p_value)

if p_value < alpha:
    print("\nwe are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables")
    print("\nwe are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs")
else:
    print("\nwe are accepting null hypothesis:  H0: MAE machine learning without time variables = MAE machine learning with time variables")


MAE without time variables: [3.16 3.04 3.17 3.08 3.11 3.15 3.05 3.07 3.13 3.15]

MAE with time variables: [3.12 3.   3.13 3.05 3.08 3.12 3.04 3.04 3.09 3.11]

alpha:  0.01

p_value point:  8.049662396968539e-07

we are rejecting null hypothesis  H0: MAE machine learning without time variables = MAE machine learning with time variables

we are accepting alternative hypothesis: the MAE of the sample with time variables inputs is less than MAE of the sample without time variables inputs


**Save algorithm**

In [9]:
import pickle
from sklearn.pipeline import Pipeline


estimators = [('ml_model', ml_model)]
pipe = Pipeline(estimators)
print(pipe)
al_file={"x_var":X.columns,"coor":coor,"pipe":pipe}
pickle.dump(al_file, open("/content/drive/MyDrive/Colab Notebooks/Coron/algorithms/sddir_coron_d0.al", 'wb'))

Pipeline(steps=[('ml_model', LGBMRegressor(n_estimators=200))])
